# An analysis of the energy intensity of the Consumer Prices Index

This analysis is an attempt to recreate a study that was carried out by the ONS:

Office for National Statistics (ONS), released 17 April 2023, ONS website, article, [The energy intensity of the Consumer Prices Index: 2022](https://www.ons.gov.uk/economy/inflationandpriceindices/articles/theenergyintensityoftheconsumerpricesindex/2022)

# import libraries

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns 

# Import ONS Data

In [6]:
# Import the energy intensity data for the CPA categories
cpa_energy_intensity = pd.read_excel('./Data/cpa to coicop conversion.xlsx', sheet_name='CPA energy intensity transposed')

# rename the cpa column so it matches the other dataframe
cpa_energy_intensity.rename(columns={'CPA':'CPA Code'}, inplace=True)

# drop description as we don't need it
cpa_energy_intensity.drop('Product', axis=1, inplace=True)

cpa_energy_intensity.head()

,CPA Code,energy intensity
0,CPA_A01,2.787781
1,CPA_A02,1.022609
2,CPA_A03,2.454325
3,CPA_B05,2.418254
4,CPA_B06 & B07,3.355954


In [7]:
# Import the ONS conversion table 
conversion = pd.read_excel('./Data/cpa to coicop conversion.xlsx', sheet_name='COICOP to CPA conversion')

conversion.head()

,COICOP Code,COICOP Description,CPA Code,CPA Description,2019 contribution
0,01.1.1,Bread and cereals,CPA_C106,"Grain mill products, starches and starch products",0.180
1,01.1.1,Bread and cereals,CPA_C107,Bakery and farinaceous products,0.579
2,01.1.1,Bread and cereals,CPA_C108,Other food products,0.241
3,01.1.2,Meat,CPA_C101,Preserved meat and meat products,1.000
4,01.1.3,Fish,CPA_A03,Fish and other fishing products; aquaculture p...,0.157


# Calculate energy intensity for COICOP categories

In [8]:
# merge the dataframes so we can get the energy intensity data into the conversion table
df = pd.merge(conversion, cpa_energy_intensity, on='CPA Code', how='left')

# Rename columns for clarity
df.rename(columns={'energy intensity':'Energy Intensity of CPA category', '2019 contribution':'CPA contribution to COICOP (2019)'}, inplace=True)

df.head()

# add another column to calculate the energy intensity contribition from each CPA
df['Contribution to energy intensity from CPA'] = df['CPA contribution to COICOP (2019)'] * df['Energy Intensity of CPA category']

df.head(100)

,COICOP Code,COICOP Description,CPA Code,CPA Description,CPA contribution to COICOP (2019),Energy Intensity of CPA category,Contribution to energy intensity from CPA
0,01.1.1,Bread and cereals,CPA_C106,"Grain mill products, starches and starch products",0.180,1.428316,0.257097
1,01.1.1,Bread and cereals,CPA_C107,Bakery and farinaceous products,0.579,1.792699,1.037973
2,01.1.1,Bread and cereals,CPA_C108,Other food products,0.241,1.233790,0.297343
3,01.1.2,Meat,CPA_C101,Preserved meat and meat products,1.000,1.363586,1.363586
4,01.1.3,Fish,CPA_A03,Fish and other fishing products; aquaculture p...,0.157,2.454325,0.385329
...,...,...,...,...,...,...,...
95,06.1.1,Pharmaceutical products,CPA_O84,Public administration and defence services; co...,0.045,1.057661,0.047595
96,06.1.1,Pharmaceutical products,CPA_C21,Basic pharmaceutical products and pharmaceutic...,0.955,0.641093,0.612243
97,06.1.2,Other medical products,CPA_C22,Rubber and plastic products,0.700,2.362760,1.653932
98,06.1.2,Other medical products,CPA_C32,Other manufactured goods,0.300,0.576766,0.173030


In [9]:
# compute the sum of the different energy intensity contributions from the various CPA categories 
# in order to get the total energy intensities of the COICOP categories

result = df.groupby('COICOP Code')['Contribution to energy intensity from CPA'].sum()

# The result is a pandas series so lets convert in into a dataframe
coicop_energy_intensity = result.to_frame()

# rename column
coicop_energy_intensity.rename(columns={'Contribution to energy intensity from CPA':'Energy Intensity'}, inplace=True)

# turn index into a column so we can access the data
coicop_energy_intensity = coicop_energy_intensity.reset_index()

coicop_energy_intensity.head(100)

,COICOP Code,Energy Intensity
0,12.4,0.094999
1,12.7,0.911667
2,01.1.1,1.592413
3,01.1.2,1.363586
4,01.1.3,1.428727
...,...,...
95,12.1.1,0.655536
96,12.1.2,0.482661
97,12.1.3,0.965776
98,12.2,0.655536


In [10]:
# Find the average energy intensity for each coicop category
coicop_category_intensity = coicop_energy_intensity.groupby(coicop_energy_intensity['COICOP Code'].str[:2]).mean()

coicop_category_intensity = coicop_category_intensity.reset_index()

print(coicop_category_intensity)


   COICOP Code  Energy Intensity
0           01          1.552524
1           02          0.128219
2           03          0.729398
3           04         15.010242
4           05          1.012506
5           06          0.966809
6           07          8.350660
7           08          1.300010
8           09          0.958859
9           10          0.906147
10          11          2.284937
11          12          0.398770


/var/folders/t7/n_c5tq1n0h985pbhlhf2w0c80000gn/T/ipykernel_41824/1377313516.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  coicop_category_intensity = coicop_energy_intensity.groupby(coicop_energy_intensity['COICOP Code'].str[:2]).mean()


In [11]:
# add the list of coicop categories

coicop_categories = ['Food and non-alcoholic beverages', 
                     'Alcoholic beverages, tobacco and narcotics', 
                     'Clothing and footwear',
                     'Housing, water, electricity, gas and other fuels',
                     'Furnishings, household equipment and routine household maintenance',
                     'Health',
                     'Transport',
                     'Communication',
                     'Recreation and culture',
                     'Education',
                     'Restaurants and hotels',
                     'Miscellaneous goods and services'
]

In [12]:
# add category names to dataframe
coicop_category_intensity['COICOP Category names'] = coicop_categories

# move columns around
e = coicop_category_intensity.pop('Energy Intensity')
coicop_category_intensity['Energy Intensity'] = e

coicop_category_intensity.head(20)

,COICOP Code,COICOP Category names,Energy Intensity
0,01,Food and non-alcoholic beverages,1.552524
1,02,"Alcoholic beverages, tobacco and narcotics",0.128219
2,03,Clothing and footwear,0.729398
3,04,"Housing, water, electricity, gas and other fuels",15.010242
4,05,"Furnishings, household equipment and routine h...",1.012506
5,06,Health,0.966809
6,07,Transport,8.350660
7,08,Communication,1.300010
8,09,Recreation and culture,0.958859
9,10,Education,0.906147


# Further Analysis

In the ONS study the team also took the energy used in distribution into account when calculating the energy intensity of the CPA categories. They used the Distributors' Trading Margins (DTMs) found in the supply and use tables to estimate these. I am not currently sure how this was achieved but I have emailed the team to clarify their method. Currently my values for the energy intensity for the various COICOP categories are similar to the ONS values, but there is a bit of variation.